# Leash Bio

- 005をChemBEATaに変更
- 各building blockごとにembedingして特徴量として使用

## ref
- https://www.kaggle.com/code/yyyu54/pytorch-version-belka-1dcnn-starter-with-all-data
- https://www.kaggle.com/code/ahmedelfazouan/belka-1dcnn-starter-with-all-data/notebook

In [7]:
exp_no = '007'
DEBUG = False
data_ratio = 1/5

In [8]:
import gc
import os
import pickle
import random
import joblib
import pandas as pd
# import polars as pd
from tqdm import tqdm

import numpy as np
from sklearn.metrics import average_precision_score as APS
from sklearn.model_selection import StratifiedKFold

import torch
from torch.utils.data import TensorDataset, Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks.lr_monitor import LearningRateMonitor

from pytorch_lightning import LightningModule
from pytorch_lightning import LightningDataModule, Trainer
# seed_everything
from pytorch_lightning.callbacks import (
    ModelCheckpoint, 
    EarlyStopping,
    TQDMProgressBar,
    LearningRateMonitor,
    ModelCheckpoint,
    RichModelSummary,
    RichProgressBar,
)
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

import warnings
warnings.simplefilter('ignore')

from funcs.tokenize import tokenize_ChemBEATa

In [9]:
import os
from pathlib import Path

def is_kaggle_kernel():
    return os.path.exists('/kaggle/working')

if is_kaggle_kernel():

    BASE_DIR = Path("/kaggle")
    DATA_DIR = BASE_DIR / "input"
    OUTPUT_DIR = BASE_DIR / "working"
    print('on kaggle notebook')

else:
    BASE_DIR = Path(os.getcwd()) / './../'
    DATA_DIR = BASE_DIR / "data"
    OUTPUT_DIR = BASE_DIR / f"output/exp{exp_no}"
    
# set device
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():    
    device = "cuda"
else:
    device = "cpu"
    
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
print('Using', torch.cuda.device_count(), 'GPU(s)')
print('pytorch:', torch.__version__)

Using 1 GPU(s)
pytorch: 2.0.0


In [10]:
class config:
    SEED = 2024
    
    PREPROCESS = False
    EPOCHS = 20 #20
    BATCH_SIZE = 4096
    NUM_WORKERS = 16
    
    LR = 1e-3
    WEIGHT_DECAY = 1e-6
    MIXED_PRECISION = True
    
    NUM_FOLDS = 5    
    USE_NUM_FOLD = 1
    
class paths:    
    DATA_DIR = DATA_DIR
    OUTPUT_DIR = OUTPUT_DIR
    MODEL_WEIGHTS_DIR = OUTPUT_DIR / f"bio-models-exp{exp_no}"
    
    SHRUNKEN_DATA_DIR = DATA_DIR / "shrunken-train-set"

    TRAIN_PATH = SHRUNKEN_DATA_DIR / "train_fold.parquet"
    TEST_PATH = SHRUNKEN_DATA_DIR / "test_fold.parquet"
    
    OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

In [11]:
print('fix seed')

def my_seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
# seed_everything(config.SEED, workers=True)
my_seed_everything(config.SEED)

fix seed


# **Loda Data**

In [12]:
bb_cols = ['buildingblock1_smiles', 'buildingblock2_smiles','buildingblock3_smiles', 'fold']
TARGETS = ['binds_BRD4', 'binds_HSA','binds_sEH']

df_train = pd.read_parquet(paths.TRAIN_PATH, columns=bb_cols + TARGETS)
    
if DEBUG:
    df_train = df_train.sample(100000).reset_index(drop=True)
else:
    len_train = int(len(df_train)*data_ratio)
    df_train = df_train.sample(len_train).reset_index(drop=True)

In [13]:
# building block smiles
# NOTE: trainとtestのindexとsmilesは一致していないっぽい
with open(paths.SHRUNKEN_DATA_DIR / 'train_dicts/BBs_dict_reverse_1.p', 'rb') as file:
    train_dicts_bb1 = pickle.load(file)
with open(paths.SHRUNKEN_DATA_DIR / 'train_dicts/BBs_dict_reverse_2.p', 'rb') as file:
    train_dicts_bb2 = pickle.load(file)
with open(paths.SHRUNKEN_DATA_DIR / 'train_dicts/BBs_dict_reverse_3.p', 'rb') as file:
    train_dicts_bb3 = pickle.load(file)

with open(paths.SHRUNKEN_DATA_DIR / 'test_dicts/BBs_dict_reverse_1_test.p', 'rb') as file:
    test_dicts_bb1 = pickle.load(file)
with open(paths.SHRUNKEN_DATA_DIR / 'test_dicts/BBs_dict_reverse_2_test.p', 'rb') as file:
    test_dicts_bb2 = pickle.load(file)
with open(paths.SHRUNKEN_DATA_DIR / 'test_dicts/BBs_dict_reverse_3_test.p', 'rb') as file:
    test_dicts_bb3= pickle.load(file)
    
# bb1のidxをscaffoldのidxに変換するdict
with open(paths.SHRUNKEN_DATA_DIR / 'test_dicts/BBs_idx_to_scaffold_idx_dict_1.p', mode='rb') as file:
    test_bb1idx2scaidx= pickle.load(file)

test_dicts_bb1_reverse = {val:key for key, val in test_dicts_bb1.items()}
test_dicts_bb2_reverse = {val:key for key, val in test_dicts_bb2.items()}
test_dicts_bb3_reverse = {val:key for key, val in test_dicts_bb3.items()}

In [14]:
df_test = pd.read_parquet(paths.DATA_DIR / 'test.parquet')
df_test.drop(['molecule_smiles'], axis=1, inplace=True)

df_test['buildingblock1_smiles'] = df_test['buildingblock1_smiles'].map(test_dicts_bb1_reverse)
df_test['buildingblock2_smiles'] = df_test['buildingblock2_smiles'].map(test_dicts_bb2_reverse)
df_test['buildingblock3_smiles'] = df_test['buildingblock3_smiles'].map(test_dicts_bb3_reverse)

# **Make Features**

In [15]:
# tokenize smiles
df_train_bb1 = tokenize_ChemBEATa(train_dicts_bb1)
df_train_bb2 = tokenize_ChemBEATa(train_dicts_bb2)
df_train_bb3 = tokenize_ChemBEATa(train_dicts_bb3)
df_test_bb1 = tokenize_ChemBEATa(test_dicts_bb1)
df_test_bb2 = tokenize_ChemBEATa(test_dicts_bb2)
df_test_bb3 = tokenize_ChemBEATa(test_dicts_bb3)

input_len = df_train_bb1.shape[1]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-10M-MLM and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.unt

# **Dataset & DataModule**

In [16]:
class BioDataset(torch.utils.data.Dataset):
    
    def __init__(
        self,
        df: pd.DataFrame,
        df_bb1: pd.DataFrame,
        df_bb2: pd.DataFrame,
        df_bb3: pd.DataFrame,
        mode = 'train'
    ):
        super().__init__()
        
        self.mode = mode
        
        meta_cols = ["buildingblock1_smiles", "buildingblock2_smiles", "buildingblock3_smiles"]
        if (self.mode == 'train') or (self.mode == 'valid'):
            meta_cols += TARGETS
            
        self.df = df[meta_cols].values
        self.bb1_array = df_bb1.values
        self.bb2_array = df_bb2.values
        self.bb3_array = df_bb3.values

        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        
        row = self.df[index, :]

        x1 = self.bb1_array[row[0], :]
        x2 = self.bb2_array[row[1], :]
        x3 = self.bb3_array[row[2], :]
        X = np.concatenate([x1, x2, x3])
        
        if (self.mode == 'train') or (self.mode == 'valid'):
            y = row[-3:]
        else:
            y = np.zeros(3)
        
        output = {
            'X': torch.tensor(X, dtype=torch.float32),
            'y': torch.tensor(y, dtype=torch.float16)
        }        
        return output

In [17]:
# Check Dataset
if DEBUG:
    dataset = BioDataset(df_train, df_train_bb1, df_train_bb2, df_train_bb3, mode='valid')
    X = dataset[0]['X']
    y = dataset[0]['y']
    print(X.shape)
    print(y.shape)

In [18]:
# lightning data module
class BioDataModule(LightningDataModule):
    def __init__(self, df_train, fold_id):
        super().__init__()
        
        self.train_df = df_train[df_train['fold'] != fold_id]
        self.valid_df = df_train[df_train['fold'] == fold_id]

    def train_dataloader(self):
        train_dataset = BioDataset(self.train_df, df_train_bb1, df_train_bb2, df_train_bb3, mode='train')
        train_dataloader = torch.utils.data.DataLoader(
                                train_dataset,
                                batch_size=config.BATCH_SIZE,
                                shuffle=True,
                                num_workers=config.NUM_WORKERS,
                                pin_memory=True,
                                persistent_workers=True,
                                drop_last=True,
                            )
        return train_dataloader

    def val_dataloader(self):
        valid_dataset = BioDataset(self.valid_df, df_train_bb1, df_train_bb2, df_train_bb3, mode='valid')
        valid_dataloader = torch.utils.data.DataLoader(
                                            valid_dataset,
                                            batch_size=config.BATCH_SIZE,
                                            shuffle=False,
                                            num_workers=config.NUM_WORKERS,
                                            pin_memory=True,
                                            persistent_workers=True,
                                            drop_last=False,
                                        )
        return valid_dataloader

# **Model**

In [19]:
class BioModel(nn.Module):
    def __init__(self, 
                 input_len_per_bb=input_len,
                 num_filters=32,
                 output_dim=3):
        super(BioModel, self).__init__()

        self.num_filters = num_filters
        self.input_len = input_len_per_bb
        self.output_dim = output_dim
        
        # embedding層は共通で持つ
        # self.embedding = nn.Embedding(num_embeddings=self.input_dim_embedding, embedding_dim=self.hidden_dim, padding_idx=0)
        
        # feature extractor 層は各building block1と2, 3で分ける
        self.feature_extractor_bb1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=self.num_filters, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv1d(in_channels=self.num_filters, out_channels=self.num_filters*2, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv1d(in_channels=self.num_filters*2, out_channels=self.num_filters*3, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1)
        )
        self.feature_extractor_bb23 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=self.num_filters, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv1d(in_channels=self.num_filters, out_channels=self.num_filters*2, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv1d(in_channels=self.num_filters*2, out_channels=self.num_filters*3, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1)
        )
        
        self.fc1 = nn.Linear(self.num_filters * 3 * 3, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.output = nn.Linear(512, self.output_dim)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        

    def forward(self, x):
        
        # feature extractor
        x1 = x[:, :self.input_len].unsqueeze(1)
        x2 = x[:, self.input_len:self.input_len*2].unsqueeze(1)
        x3 = x[:, self.input_len*2:].unsqueeze(1)

        x1 = self.feature_extractor_bb1(x1).squeeze(-1)
        x2 = self.feature_extractor_bb23(x2).squeeze(-1)
        x3 = self.feature_extractor_bb23(x3).squeeze(-1)
                
        # concat
        x = torch.cat([x1, x2, x3], dim=1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        
        x = self.output(x)
        
        return x

In [20]:
# check model
if DEBUG:
    dummy_model = BioModel()
    total_params = sum(p.numel() for p in dummy_model.parameters())
    print(f"Total number of parameters: {total_params}")
    
    dummy_input = torch.rand((64, 1152), dtype=torch.float32)
    output = dummy_model(dummy_input)

# **Lightning Module**

In [21]:
def calc_score(y_preds, y_true):
    score_BRD4 = APS(y_true[:,0], y_preds[:,0])
    score_HSA = APS(y_true[:,1], y_preds[:,1])
    score_sEH = APS(y_true[:,2], y_preds[:,2])
    score = (score_BRD4 + score_HSA + score_sEH) / 3
    
    return score_BRD4, score_HSA, score_sEH, score

In [22]:
class BioModule(LightningModule):
    def __init__(self):
        
        super(BioModule, self).__init__()
       
        self.model = BioModel()
        self.validation_step_outputs = []
        self.loss_func = nn.BCEWithLogitsLoss()
        
    def forward(self, X):
        pred = self.model(X)
        return pred
    
    def configure_optimizers(self):
        
        # == define optimizer ==
        model_optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, self.parameters()),
            lr=config.LR,
            weight_decay=config.WEIGHT_DECAY
        )
        # == define learning rate scheduler ==
        lr_scheduler = CosineAnnealingWarmRestarts(
            model_optimizer,
            T_0=config.EPOCHS,
            T_mult=1,
            eta_min=1e-6,
            last_epoch=-1
        )
        return {
            'optimizer': model_optimizer,
            'lr_scheduler': {
                'scheduler': lr_scheduler,
                'interval': 'epoch',
                'monitor': 'valid_loss_epoch',
                'frequency': 1
            }
        }
        
    def training_step(self, batch, batch_idx):
        
        X, y = batch.pop('X'), batch.pop('y')
        logits = self(X)
        train_loss = self.loss_func(logits, y)
        
        self.log('train_loss', train_loss,  on_step=True, on_epoch=True, prog_bar=True, logger=True, batch_size=X.size(0))
        
        return train_loss

    def validation_step(self, batch, batch_idx):
        
        X, y = batch.pop('X'), batch.pop('y')
        logits = self(X)
        preds = torch.sigmoid(logits)
        valid_loss = self.loss_func(logits, y)
        
        self.log('valid_loss', valid_loss, on_step=True, on_epoch=False, prog_bar=True, logger=True, batch_size=X.size(0))
        
        self.validation_step_outputs.append({"valid_loss":valid_loss, "preds":preds, "targets":y})
        
        return valid_loss

    
    def train_dataloader(self):
        return self._train_dataloader

    def validation_dataloader(self):
        return self._validation_dataloader
    
    def calc_score(self, y_preds, y_true):
        return calc_score(y_preds, y_true)
    
    def on_validation_epoch_end(self):
        
        outputs = self.validation_step_outputs
        
        # 各iterationごとのlossを平均
        avg_loss = torch.stack([x['valid_loss'] for x in outputs]).mean()
        self.log("valid_loss_epoch", avg_loss, prog_bar=True, logger=True)
        
        # scoreを計算
        y_preds = torch.cat([x['preds'] for x in outputs],dim=0).detach().cpu().numpy()
        y_true = torch.cat([x['targets'] for x in outputs],dim=0).detach().cpu().numpy()

        score = self.calc_score(y_preds, y_true)[-1]
        self.log("valid_score", score, prog_bar=True, logger=True)
        
        self.validation_step_outputs.clear()
        
        return {'valid_loss_epoch': avg_loss, "valid_score":score}

# Train & Inference

In [28]:
def find_latest_ckpt_path(fold):
    ckpt_list = list(paths.MODEL_WEIGHTS_DIR.glob(f'fold_{fold}*.ckpt'))

    latest_ckpt = max(ckpt_list, key=lambda p: p.stat().st_mtime)
    print(f'Latest checkpoint file: {latest_ckpt}')
    return latest_ckpt


def old_ckpt_path(fold):
    # Assuming paths.CKPT_ROOT and fold_num are already defined
    ckpt_list = list(paths.MODEL_WEIGHTS_DIR.glob(f'fold_{fold}*.ckpt'))
    print(f'find {len(ckpt_list)} ckpts')
    print(ckpt_list)

    # Find the latest checkpoint file
    if ckpt_list:
        latest_ckpt = max(ckpt_list, key=lambda p: p.stat().st_mtime)
        print(f'Latest checkpoint file: {latest_ckpt}')

        # Delete all other checkpoint files
        for ckpt in ckpt_list:
            if ckpt != latest_ckpt:
                print(f'Deleting checkpoint file: {ckpt}')
                ckpt.unlink()
    else:
        print('No checkpoint files found')
    


def predict_in_batches(model, df, df_bb1, df_bb2, df_bb3, mode):
    
    model.to(device)
    model.eval()
    
    dataset = BioDataset(df, df_bb1, df_bb2, df_bb3, mode=mode)
    dataloader = torch.utils.data.DataLoader(
                                        dataset,
                                        batch_size=config.BATCH_SIZE,
                                        shuffle=False,
                                        num_workers=config.NUM_WORKERS,
                                        pin_memory=True,
                                        persistent_workers=True,
                                        drop_last=False,
                                    )

    all_preds = []
    with torch.no_grad():
        for batch in dataloader:
            inputs = batch['X'].to(device)
            logits = model(inputs)
            preds = torch.sigmoid(logits)
            all_preds.append(preds.cpu().numpy())
    
    return np.concatenate(all_preds, axis=0)

In [24]:
def run_training(fold_id, df, infer=False):
    print(f"======== Running training for fold {fold_id} =============")
    
    # == init data module and model ==
    model = BioModule()
    datamodule = BioDataModule(df, fold_id)
    
    # == init callback ==
    checkpoint_callback = ModelCheckpoint(
                                        monitor='valid_score',
#                                             monitor='valid_loss_epoch',
                                          dirpath=paths.MODEL_WEIGHTS_DIR,
                                          save_top_k=1,
                                          save_last=False,
                                          save_weights_only=True,
                                          filename=f"fold_{fold_id}",
                                          mode='max'
                                          )
    early_stop_callback = EarlyStopping(
        monitor='valid_score',
#         monitor="valid_loss_epoch", 
        mode="max", 
        patience=5,
        verbose=True
        )
    callbacks_to_use = [checkpoint_callback,
                        early_stop_callback,
                        RichModelSummary(),
                        RichProgressBar(),
                       ]

    # == init trainer ==
    trainer = Trainer(
        max_epochs=config.EPOCHS,
        callbacks=callbacks_to_use,
        accelerator=device,
        devices=-1,  # 全ての利用可能なGPUを使用
        deterministic=False,
        precision='16-mixed' if config.MIXED_PRECISION else 32,
        logger=TensorBoardLogger('lightning_logs', name=f'exp{exp_no}_fold{fold_id}'),
    )
    
    if not infer:
        # == Training ==
        trainer.fit(model, datamodule=datamodule)
        weights = torch.load(checkpoint_callback.best_model_path)['state_dict']
    else:
        ckpt_path = find_latest_ckpt_path(fold_id) 
        weights = torch.load(ckpt_path)['state_dict']
        
    model.load_state_dict(weights)
    
    valid_df = datamodule.valid_df
    
    preds_oof = predict_in_batches(model, valid_df, df_train_bb1, df_train_bb2, df_train_bb3, mode='valid')
    y_oof = valid_df[TARGETS].values
    
    score_BRD4, score_HSA, score_sEH, score = calc_score(preds_oof, y_oof)
    
    valid_df[[f'{target}_pred' for target in TARGETS]] = preds_oof
    
    print(f'fold:{fold_id} | CV score = {score}')
    
    df_test_temp = df_test.drop(['id'], axis=1)
    preds_test = predict_in_batches(model, df_test_temp, df_test_bb1, df_test_bb2, df_test_bb3, mode='test')
    
    del model, datamodule, trainer, preds_oof, y_oof
    gc.collect()
    
    score_dict = {
        'BRD4':score_BRD4,
        "HSA":score_HSA,
        "sEH":score_sEH,
        "all":score
    }
    
    return preds_test, score_dict, valid_df

In [25]:
# training
# torch.set_float32_matmul_precision('high')

# tokenizerの warning対策
os.environ["TOKENIZERS_PARALLELISM"] = "false"

all_preds = []
score_list = []
score_list_BRD4 = []
score_list_HSA = []
score_list_sEH = []

def save_list_by_text(score_list, filename):
    # ファイルに書き込み
    score_list_txt = [str(loss) for loss in score_list]
    with open(paths.OUTPUT_DIR / f'{filename}.txt', 'w') as file:
        file.write(', '.join(score_list_txt))
    

for fold_id in range(config.NUM_FOLDS):
    
    preds_test, score_dict, df_oof = run_training(fold_id, df_train, infer=True)
    
    # save score
    score_list_BRD4.append(score_dict['BRD4'])
    score_list_HSA.append(score_dict['HSA'])
    score_list_sEH.append(score_dict['sEH'])
    score_list.append(score_dict['all'])
    
    save_list_by_text(score_list, 'cv_all')
    save_list_by_text(score_list_BRD4, 'cv_BRD4')
    save_list_by_text(score_list_HSA, 'cv_HSA')
    save_list_by_text(score_list_sEH, 'cv_sEH')
    
    # save preds（foldごと）
    all_preds.append(preds_test) 
    
    df_oof.to_parquet(paths.OUTPUT_DIR / f"oof_fold_{fold_id}.parquet")
    
    del df_oof
    gc.collect()
    

df_oof_all = pd.DataFrame()
for fold_id in range(config.NUM_FOLDS):
    df_temp = pd.read_parquet(paths.OUTPUT_DIR / f"oof_fold_{fold_id}.parquet")
    df_oof_all = pd.concat([df_oof_all, df_temp], axis=0)
    
df_oof_all.to_parquet(paths.OUTPUT_DIR / f"oof_all.parquet")

======== Running training for fold 0 =============


Using 16bit Automatic Mixed Precision (AMP)
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Latest checkpoint file: /home/working/notebooks/../output/exp007/bio-models-exp007/fold_0-v13.ckpt
fold:0 | CV score = 0.21401210723799427


/tmp/ipykernel_2463081/2156084032.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df[[f'{target}_pred' for target in TARGETS]] = preds_oof
/tmp/ipykernel_2463081/2156084032.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df[[f'{target}_pred' for target in TARGETS]] = preds_oof
/tmp/ipykernel_2463081/2156084032.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

======== Running training for fold 1 =============


...

======== Running training for fold 4 =============


Using 16bit Automatic Mixed Precision (AMP)
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Latest checkpoint file: /home/working/notebooks/../output/exp007/bio-models-exp007/fold_4-v1.ckpt
fold:4 | CV score = 0.2848297637641219


/tmp/ipykernel_2463081/2156084032.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df[[f'{target}_pred' for target in TARGETS]] = preds_oof
/tmp/ipykernel_2463081/2156084032.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df[[f'{target}_pred' for target in TARGETS]] = preds_oof
/tmp/ipykernel_2463081/2156084032.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

NameError: name 'df_oof' is not defined

In [26]:

df_oof_all = pd.DataFrame()
for fold_id in range(config.NUM_FOLDS):
    df_temp = pd.read_parquet(paths.OUTPUT_DIR / f"oof_fold_{fold_id}.parquet")
    df_oof_all = pd.concat([df_oof_all, df_temp], axis=0)
    df_oof_all.to_parquet(paths.OUTPUT_DIR / f"oof_all.parquet")

# **Submission**

In [27]:
preds = np.mean(all_preds, 0)

df_test['binds'] = 0
df_test.loc[df_test['protein_name']=='BRD4', 'binds'] = preds[df_test['protein_name']=='BRD4', 0]
df_test.loc[df_test['protein_name']=='HSA', 'binds'] = preds[df_test['protein_name']=='HSA', 1]
df_test.loc[df_test['protein_name']=='sEH', 'binds'] = preds[df_test['protein_name']=='sEH', 2]
df_test[['id', 'binds']].to_csv(paths.OUTPUT_DIR / f'submission_fold{fold_id}.csv', index = False)

In [ ]:

for fold in range(0, 5):    
    # 古いckpt pathを削除
    old_ckpt_path(fold)
